In [15]:
from PIL import Image
import numpy as np

def text_to_bin(text):
    """Convert text to binary."""
    binary = ''.join([format(ord(i), '08b') for i in text])
    return binary

def bin_to_text(binary):
    """Convert binary to text."""
    text = ''.join([chr(int(binary[i:i+8], 2)) for i in range(0, len(binary), 8)])
    return text

def encode_image(image_path, text, output_path):
    # Load image
    image = Image.open(image_path)
    image_data = np.array(image)
    
    # Convert text to binary
    binary_text = text_to_bin(text)
    binary_text += '1111111111111110'  # Add a delimiter to mark the end of the text
    
    data_index = 0
    text_len = len(binary_text)
    
    # Handle both grayscale and RGB images
    if image_data.ndim == 2:  # Grayscale image
        for row in range(image_data.shape[0]):
            for col in range(image_data.shape[1]):
                if data_index < text_len:
                    image_data[row, col] = (image_data[row, col] & ~1) | int(binary_text[data_index])
                    data_index += 1
                if data_index >= text_len:
                    break
            if data_index >= text_len:
                break
    elif image_data.ndim == 3:  # RGB image
        for row in range(image_data.shape[0]):
            for col in range(image_data.shape[1]):
                for color in range(3):  # Iterate over R, G, B
                    if data_index < text_len:
                        image_data[row, col, color] = (image_data[row, col, color] & ~1) | int(binary_text[data_index])
                        data_index += 1
                    if data_index >= text_len:
                        break
                if data_index >= text_len:
                    break
            if data_index >= text_len:
                break
    
    # Save the modified image
    encoded_image = Image.fromarray(image_data)
    encoded_image.save(output_path)

def decode_image(image_path):
    # Load image
    image = Image.open(image_path)
    image_data = np.array(image)
    
    binary_text = ''
    
    # Handle both grayscale and RGB images
    if image_data.ndim == 2:  # Grayscale image
        for row in range(image_data.shape[0]):
            for col in range(image_data.shape[1]):
                binary_text += str(image_data[row, col] & 1)
    elif image_data.ndim == 3:  # RGB image
        for row in range(image_data.shape[0]):
            for col in range(image_data.shape[1]):
                for color in range(3):
                    binary_text += str(image_data[row, col, color] & 1)
    
    # Find the delimiter to determine the end of the message
    end_delimiter = '1111111111111110'
    end_index = binary_text.find(end_delimiter)
    if end_index != -1:
        binary_text = binary_text[:end_index]
    
    # Convert binary to text
    text = bin_to_text(binary_text)
    return text

# Example usage
input_image_path = 'pendant-lamp-over-river.png'
output_image_path = 'encoded_image.png'
text_to_hide = "Hello, this is a secret message!"

# Encode the text into the image
encode_image(input_image_path, text_to_hide, output_image_path)

# Decode the text from the image
decoded_text = decode_image(output_image_path)
print("Decoded text:", decoded_text)


Decoded text: Hello, this is a secret message!
